In [8]:
import requests
from bs4 import BeautifulSoup
import spacy

def fetch_historic_facts_v2(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        content_section = soup.find("div", {"class": "detail"})


        if content_section:
            text = content_section.get_text()
            
            nlp = spacy.load("fr_core_news_sm")
            doc = nlp(text)

            historic_facts = []

            for ent in doc.ents:
                if ent.label_ in ["DATE", "EVENT"]:
                    sentence = ent.sent.text.strip()
                    if sentence not in historic_facts:
                        historic_facts.append(sentence)

            return historic_facts
        else:
            print("La section de contenu n'a pas été trouvée sur la page.")
            return []
    else:
        print(f"Erreur lors de la récupération de la page. Code d'erreur : {response.status_code}")
        return []

url = "https://www.lagarennecolombes.fr/Histoire%20et%20patrimoine/6042/6164"
historic_facts = fetch_historic_facts_v2(url)



In [10]:
import re, requests
from bs4 import BeautifulSoup
url = "https://www.lagarennecolombes.fr/Histoire%20et%20patrimoine/6042/6164"
def extract_historic_facts_regex(text):
    # Expression régulière pour identifier les dates et les événements historiques
    # Expression régulière pour identifier les dates
    date_pattern = r"(\b(?:\d{1,4}|X{1,3}I{1,3}|X{1,3}V?I{0,3}|V?I{1,3})\s?(?:siècle|année|époque)\b)"

    # Expression régulière pour identifier les lieux (rues, places, avenues, boulevards)
    location_pattern = r"(\b(?:rue|place|avenue|boulevard)[^\n,.!?]*\b)"

    # Combinaison des expressions régulières
    regex = re.compile(f"({date_pattern}|{location_pattern})", re.IGNORECASE)

    # Découper le texte en phrases en utilisant un point, un point d'interrogation ou un point d'exclamation comme séparateur
    sentences = re.split(r"[.!?]", text)
    # Trouver les phrases qui correspondent à l'expression régulière
    historic_facts = [sentence.strip() for sentence in sentences if regex.search(sentence)]

    return historic_facts

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    content_section = soup.find("div", {"class": "detail"})
    if content_section:
        text = content_section.get_text()
historic_facts = extract_historic_facts_regex(text)

for fact in historic_facts:
    print(fact)


Le début du XVIe siècle voit la construction de sa première habitation : La Tournelle, située sur l'actuelle place de Belgique, demeure du fermier
L'urbanisation transforme alors sa physionomie d'origine; des voies et places sont tracées et, le 31 décembre 1886, les architectes Jean-Baptiste Lafolie et Guerton remettent les plans de cette nouvelle propriété communale au maire de Colombes


In [4]:
import spacy
import openai

# Chargement du modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")

# Texte d'exemple contenant des faits historiques
text = """La rue Saint-Honoré est une voie du 1er arrondissement de Paris, en France. Elle doit son nom à l'église Saint-Honoré. Elle fut tracée pour la première fois au début du XIIIe siècle. En 1564, Catherine de Médicis fit construire le palais des Tuileries à l'ouest de la rue."""

# Traitement du texte avec spaCy
doc = nlp(text)

# Extraction des entités nommées
entities = {ent.text: ent.label_ for ent in doc.ents}

# Générer un résumé des faits historiques à l'aide de GPT-4
# Générer un résumé des faits historiques à l'aide de GPT-4
response = openai.Completion.create(
    engine="davinci",  # Utiliser l'alias du modèle GPT-3 Davinci pour un résumé de texte
    prompt=f"Résumé des faits historiques pour le texte suivant : '{text}'",
    temperature=0.5,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Afficher le résumé des faits historiques généré par GPT-3
print("Résumé des faits historiques :")
print(response.choices[0].text.strip())




Résumé des faits historiques :
Résumé des faits historiques pour le texte suivant : 'La rue Saint-Honoré est une voie du 1er arrondissement de Paris, en France. Elle doit son nom à l'église Saint-Honoré. Elle fut tracée pour la première fois au début du XIIIe siècle. En 1564, Catherine de Médicis fit construire


In [3]:
import spacy
from transformers import pipeline

# Chargement du modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")

# Création d'une pipeline de résumé avec Hugging Face Transformers
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")

# Texte d'exemple contenant des faits historiques
text = """La rue Saint-Honoré est une voie du 1er arrondissement de Paris, en France. Elle doit son nom à l'église Saint-Honoré. Elle fut tracée pour la première fois au début du XIIIe siècle. En 1564, Catherine de Médicis fit construire le palais des Tuileries à l'ouest de la rue."""

# Traitement du texte avec spaCy
doc = nlp(text)

# Extraction des entités nommées
entities = {ent.text: ent.label_ for ent in doc.ents}

# Générer un résumé des faits historiques à l'aide du modèle BART
summary = summarizer(text, max_length=150)

# Afficher les faits historiques extraits
print("Faits historiques :")
print(summary[0]["summary_text"])


Your max_length is set to 150, but you input_length is only 100. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Faits historiques :
Elle doit son nom à l'église Saint-Honoré. Elle fut tracée pour la première fois au début du XIIIe siècle. En 1564, Catherine de Médicis fit construire le palais des Tuileries.


In [35]:
import json
cities = json.load(open("citiesv2.json", "r", encoding="utf-8"))

In [ ]:
from googlesearch import search
count = 0

for city in cities:
    if count < 10:
        if city["name"] != "Marseille":
            query = f"histoire de {city['name']}"
            print(query)
            for j in search(query, num_results=1):
                print(j)
            
            count += 1

In [ ]:
count = 0
for city in cities:
    for street in city["streets"]:
        if street["wikipediaUrl"] is not None:
            print(street["wikipediaUrl"])
            count += 1